In [1]:
# EE104 HW7
using JSON
using LinearAlgebra
using Plots
using Flux
using Random

include("multi_logistic.jl")
include("readclassjson.jl")

data1 = readclassjson("beer_predictions.json")
data2 = readclassjson("multi_class.json")

v_true = data1["v_true"]
y_hat = data1["y_hat"]

U_train = data2["U_train"]
v_train = data2["v_train"]
U_test = data2["U_test"]
v_test = data2["v_test"]

300-element Array{Int64,1}:
  1
  1
 -1
  1
 -1
  0
  1
  1
 -1
 -1
  1
  0
  0
  ⋮
  0
 -1
  1
  1
  1
  1
  0
 -1
  0
  0
 -1
  0

In [2]:
# Question 1 

# Part a
@show v_true
@show length(unique(v_true))

v_true = ["pilsner", "saison", "pilsner", "hefeweizen", "IPA", "stout", "pilsner", "saison", "pilsner", "hefeweizen", "saison", "hefeweizen", "saison", "stout", "amber", "hefeweizen", "pilsner", "stout", "stout", "pilsner", "IPA", "amber", "saison", "amber", "saison", "IPA", "IPA", "stout", "amber", "amber", "saison", "saison", "hefeweizen", "amber", "stout", "IPA", "amber", "stout", "amber", "amber", "pilsner", "pilsner", "hefeweizen", "stout", "stout", "pilsner", "saison", "pilsner", "pilsner", "saison", "saison", "hefeweizen", "saison", "stout", "stout", "IPA", "IPA", "pilsner", "amber", "saison", "pilsner", "stout", "amber", "saison", "pilsner", "hefeweizen", "amber", "IPA", "hefeweizen", "hefeweizen", "pilsner", "IPA", "amber", "stout", "hefeweizen", "stout", "saison", "stout", "stout", "hefeweizen", "amber", "pilsner", "hefeweizen", "IPA", "hefeweizen", "IPA", "hefeweizen", "hefeweizen", "pilsner", "pilsner", "saison", "IPA", "saison", "stout", "saison", "pilsner", "IPA", "stout"

length(unique(v_true)) = 6


6

From the set, we see that V = {stout, amber, IPA, pilsner, saison, hefeweizen}, so K = 6 for v_true. 

In [3]:
# Part b
@show y_hat

y_hat = [1, 3, 2, 6, 1, 3, 1, 4, 1, 6, 4, 6, 4, 3, 5, 6, 2, 3, 3, 2, 1, 5, 4, 5, 4, 1, 1, 3, 5, 5, 3, 4, 6, 4, 3, 6, 4, 3, 5, 5, 2, 2, 6, 2, 3, 2, 4, 2, 2, 4, 4, 6, 4, 3, 3, 1, 1, 2, 5, 4, 1, 2, 5, 4, 2, 5, 5, 6, 6, 5, 2, 1, 5, 3, 6, 3, 4, 3, 3, 6, 5, 2, 6, 1, 6, 1, 6, 6, 1, 2, 4, 6, 3, 2, 3, 2, 6, 3, 2, 2, 6, 1, 2, 6, 6, 3, 4, 1, 1, 5, 3, 3, 3, 6, 6, 2, 2, 2, 5, 2, 4, 6, 3, 3, 6, 1, 2, 4, 2, 5, 1, 4, 6, 6, 5, 5, 3, 4, 2, 5, 3, 2, 1, 6, 1, 1, 2, 4, 6, 4, 4, 4, 6, 5, 2, 6, 4, 1, 5, 3, 6, 6, 2, 6, 1, 3, 2, 5, 2, 3, 5, 2, 6, 3, 6, 3, 4, 5, 5, 6, 3, 1, 1, 1, 2, 4, 1, 4, 5, 4, 3, 6, 6, 4, 3, 1, 4, 1, 6, 4, 5, 6, 6, 2, 1, 6, 4, 2, 4, 4, 5, 1, 6, 5, 1, 6, 3, 4, 4, 1, 3, 3, 5, 1, 6, 1, 1, 2, 1, 4, 3, 2, 2, 3, 6, 5, 2, 4, 4, 5, 2, 6, 4, 5, 3, 6, 5, 2, 6, 6, 5, 2, 6, 6, 5, 4, 3, 5, 6, 1, 3, 1, 5, 1, 6, 6, 4, 3, 3, 5, 2, 1, 3, 1, 4, 1, 1, 1, 1, 2, 5, 5, 4, 3, 2, 2, 4, 2, 1, 4, 3, 3, 4, 2, 3, 1, 5, 2, 3, 6, 6, 6, 6, 3, 5, 6, 5, 4, 6, 2, 3, 4, 4, 2, 3, 1, 6, 1, 5, 6, 2, 6, 5, 2, 5, 5, 6, 5, 3, 5, 6

1000-element Array{Int64,1}:
 1
 3
 2
 6
 1
 3
 1
 4
 1
 6
 4
 6
 4
 ⋮
 6
 2
 4
 4
 2
 2
 6
 5
 5
 2
 2
 5

From the data above, I recommend the following embedding function:
IPA -> 1 ;
pilsner -> 2 ;
stout -> 3 ;
saison -> 4 ;
amber -> 5 ;
hefeweizen -> 6 ;
Note that the unembedding function will simply be the inverse. We simply take the string associated with each of the numbers.

In [4]:
# Part c
v_hat = Array{Union{Missing, String}}(missing, length(y_hat))

# Unembed input data 
for i = 1 : length(y_hat)
    if y_hat[i] == 1
        v_hat[i] = "IPA"
    elseif y_hat[i] == 2
        v_hat[i] = "pilsner"
    elseif y_hat[i] == 3
        v_hat[i] = "stout"
    elseif y_hat[i] == 4
        v_hat[i] = "saison"
    elseif y_hat[i] == 5
        v_hat[i] = "amber"
    else
        v_hat[i] = "hefeweizen"
    end
end

# Embed output data
y_true = zeros(size(v_true))

for j = 1 : length(y_hat)
    if v_true[j] == "IPA"
        y_true[j] = 1
    elseif v_true[j] == "pilsner"
        y_true[j] = 2
    elseif v_true[j] == "stout"
        y_true[j] = 3
    elseif v_true[j] == "saison"
        y_true[j] = 4
    elseif v_true[j] == "amber"
        y_true[j] = 5
    else
        y_true[j] = 6
    end
end

# Create and report confusion matrix
CM = [0 0 0 0 0 0; 0 0 0 0 0 0; 0 0 0 0 0 0; 0 0 0 0 0 0; 0 0 0 0 0 0; 0 0 0 0 0 0]

for k = 1 : length(y_hat)
    if v_hat[k] == v_true[k]
        CM[convert(Int64, y_hat[k]), convert(Int64, y_hat[k])] = CM[convert(Int64, y_hat[k]), convert(Int64, y_hat[k])] + 1
    else
        CM[convert(Int64, y_true[k]), convert(Int64, y_hat[k])] = CM[convert(Int64, y_true[k]), convert(Int64, y_hat[k])] + 1 
    end
end

@show CM
error_rate =  (CM[1,6] + CM[2,1] + CM[3,2] + CM[4,3] + CM[5,4] + CM[6,5]) / length(y_hat)
@show error_rate

CM = [133 0 0 0 0 30; 33 138 0 0 0 0; 0 32 117 0 0 0; 0 0 43 128 0 0; 0 0 0 32 137 0; 0 0 0 0 29 148]
error_rate = 0.199


0.199

In [5]:
# Question 2

In [6]:
@show size(unique(v_train))
@show unique(v_train)
@show size(unique(v_test))
@show unique(v_test)

k1 = 0
k2 = 0
k3 = 0

for i = 1:length(v_train)
    if v_train[i] == -1
        global k1 += 1
    elseif v_train[i] == 0
        global k2 += 1
    else
        global k3 += 1
    end
end

@show k1
@show k2
@show k3

k1_t = 0
k2_t = 0
k3_t = 0

for j = 1:length(v_test)
    if v_test[j] == -1
        global k1_t += 1
    elseif v_test[j] == 0
        global k2_t += 1
    else
        global k3_t += 1
    end
end

@show k1_t
@show k2_t
@show k3_t

size(unique(v_train)) = (3,)
unique(v_train) = [1, 0, -1]
size(unique(v_test)) = (3,)
unique(v_test) = [1, -1, 0]
k1 = 118
k2 = 68
k3 = 114
k1_t = 121
k2_t = 65
k3_t = 114


114

V = {1, 0 , -1} and K = 3. The values are reported above, where the k values sufficed with a t representing the test set. 

In [7]:
# Part b

I propose the we use the raw data as a one hot embedding, where -1 -> [1, 0, 0] ; 0 -> [0, 1, 0] ; and 1 -> [0, 0, 1]

In [11]:
# Part c

X_tr = [ones(size(U_train)[1], 1) U_train]
Y_tr = zeros(size(v_train)[1], 3)

for i = 1:size(v_train)[1]
    if v_train[i] == -1
        Y_tr[i,:] = [1 0 0]
    elseif v_train[i] == 0
        Y_tr[i, :] = [0 1 0]
    else
        Y_tr[i, :] = [0 0 1]
    end
end

X_te = [ones(size(U_test)[1], 1) U_test]
Y_te = zeros(size(v_test)[1], 3)

for j = 1:size(v_test)[1]
    if v_test[j] == -1
        Y_te[j,:] = [1 0 0]
    elseif v_test[j] == 0
        Y_te[j, :] = [0 1 0]
    else
        Y_te[j, :] = [0 0 1]
    end
end

reps = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]

g_tr = multi_logistic(X_tr, Y_tr, reps)

@show g_tr

┌ Info: Epoch 1
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 2
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 3
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 4
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 5
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 6
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 7
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 8
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 9
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 10
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 11
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\s

┌ Info: Epoch 92
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 93
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 94
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 95
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 96
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 97
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 98
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 99
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 100
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 101
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 102
└ @ Main C:\Users\thejo\.julia\packages\

┌ Info: Epoch 179
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 180
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 181
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 182
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 183
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 184
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 185
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 186
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 187
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 188
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 189
└ @ Main C:\Users\thejo\.julia\p

┌ Info: Epoch 267
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 268
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 269
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 270
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 271
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 272
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 273
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 274
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 275
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 276
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 277
└ @ Main C:\Users\thejo\.julia\p

┌ Info: Epoch 354
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 355
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 356
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 357
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 358
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 359
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 360
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 361
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 362
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 363
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 364
└ @ Main C:\Users\thejo\.julia\p

┌ Info: Epoch 444
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 445
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 446
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 447
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 448
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 449
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 450
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 451
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 452
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 453
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 454
└ @ Main C:\Users\thejo\.julia\p

g_tr = (var"#predicty#5"{Array{Float64,2}}([-0.8379030861618484 1.5800237886663404 -1.5760768802952985; -3.328153653507964 -0.02417810698213025 3.544496487451256; -2.516041127783548 0.15374280621464684 3.356574970474333; -2.6090431416498965 0.08029353161102298 3.4952567723353223; -3.026695976615664 0.5015987520322792 3.6260080158576233; -3.1345727186524646 -0.14258960193413617 3.067205902911477; -2.7982749977452395 0.13923104230173677 3.3914796189076153; -2.2434046929477662 0.016524465884370954 3.600365118127796; -0.11022428225851326 0.04584138069525481 0.16577765408301362; -0.07034888425479797 -0.3476382369072386 0.34027060941277343; -0.4110626021494095 -0.10090829587861644 0.6543554045680734; -1.0987454794923777 -0.2180142665507829 1.1605922257455952; -0.7920329572910947 -0.12471564157929389 1.0974134796008383; -0.47799909878511876 -0.22065249228864395 0.7752430370647172; -0.46926835759114127 0.1147416914820426 0.16051936538718384; -0.5703291393333436 0.008355760276972034 0.486683673

┌ Info: Epoch 499
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121
┌ Info: Epoch 500
└ @ Main C:\Users\thejo\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


(var"#predicty#5"{Array{Float64,2}}([-0.8379030861618484 1.5800237886663404 -1.5760768802952985; -3.328153653507964 -0.02417810698213025 3.544496487451256; … ; -0.13995661426718042 0.38663576196085886 -0.31259796269948587; -0.10449490343912961 -0.03389748340366578 0.10592000185641373]), [-0.8379030861618484 1.5800237886663404 -1.5760768802952985; -3.328153653507964 -0.02417810698213025 3.544496487451256; … ; -0.13995661426718042 0.38663576196085886 -0.31259796269948587; -0.10449490343912961 -0.03389748340366578 0.10592000185641373])

In [47]:
function unembed(y_hat, labs)
    smallval = norm(y_hat - labs[1])
    small = labs[1]
    for i = 2:length(labs)
        if norm(y_hat - labs[i]) < smallval
            smallval = norm(y_hat - labs[i])
            small = labs[i]
        end
    end
    return small
end

unembed (generic function with 1 method)

In [48]:
# psiinv(yhat) = reps[argmin([norm(yhat-reps[k]) for k=1:length(reps)])]
@show g_tr[1](X_tr[1,:])

(g_tr[1])(X_tr[1, :]) = [-8.371090394048895, -0.13723380980953737, 8.41565653753311]


3-element Array{Float64,1}:
 -8.371090394048895  
 -0.13723380980953737
  8.41565653753311   

In [57]:
CM_tr = [0 0 0; 0 0 0; 0 0 0]
CM_te = [0 0 0; 0 0 0; 0 0 0]

y_hat_tr = zeros(size(Y_tr)[1], 3)
for i = 1:size(Y_tr)[1]
    p_tr = g_tr[1](X_tr[i,:])
    y_hat_tr[i, :] = unembed(p_tr, reps)
    if y_hat_tr[i,:] == Y_tr[i,:]
        if y_hat_tr[i,:] == [1, 0, 0]
            CM_tr[1,1] += 1
        elseif y_hat_tr[i,:] == [0, 1, 0]
            CM_tr[2,2] += 1
        else
            CM_tr[3,3] += 1
        end
    else
        if y_hat_tr[i,:] == [1, 0, 0]
            if Y_tr[i,:] == [0, 1, 0]
                CM_tr[2, 1] += 1 
            else
                CM_tr[3, 1] += 1
            end
        elseif y_hat_tr[i,:] == [0, 1, 0]
            if Y_tr[i,:] == [1, 0, 0]
                CM_tr[1, 2] += 1 
            else
                CM_tr[3, 2] += 1
            end
        else
            if Y_tr[i,:] == [1, 0, 0]
                CM_tr[1, 3] += 1 
            else
                CM_tr[2, 3] += 1
            end
        end
    end
end

@show CM_tr

y_hat_te = zeros(size(Y_te)[1], 3)
for i = 1:size(Y_te)[1]
    p_te = g_tr[1](X_te[i,:])
    y_hat_te[i, :] = unembed(p_te, reps)
    if y_hat_te[i,:] == Y_te[i,:]
        if y_hat_te[i,:] == [1, 0, 0]
            CM_te[1,1] += 1
        elseif y_hat_te[i,:] == [0, 1, 0]
            CM_te[2,2] += 1
        else
            CM_te[3,3] += 1
        end
    else
        if y_hat_te[i,:] == [1, 0, 0]
            if Y_te[i,:] == [0, 1, 0]
                CM_te[2, 1] += 1 
            else
                CM_te[3, 1] += 1
            end
        elseif y_hat_te[i,:] == [0, 1, 0]
            if Y_te[i,:] == [1, 0, 0]
                CM_te[1, 2] += 1 
            else
                CM_te[3, 2] += 1
            end
        else
            if Y_te[i,:] == [1, 0, 0]
                CM_te[1, 3] += 1 
            else
                CM_te[2, 3] += 1
            end
        end
    end
end


@show CM_te

error_rate_tr = 1 - ((CM_tr[1,1] + CM_tr[2,2] + CM_tr[3,3]) / size(Y_tr)[1])
error_rate_te = 1 - ((CM_te[1,1] + CM_te[2,2] + CM_te[3,3]) / size(Y_te)[1])
@show error_rate_tr
@show error_rate_te

CM_tr = [112 6 0; 5 58 5; 0 3 111]
CM_te = [110 10 1; 15 37 13; 0 13 101]
error_rate_tr = 0.06333333333333335
error_rate_te = 0.17333333333333334


0.17333333333333334